In [2]:
import pandas as pd
import matplotlib.pyplot as plt

sentiment_type = 'positive_sentiment'


# Assuming sentiment_scores is already a list of floats and split into three columns
tweets_df = pd.read_csv('./sentiment_scores.csv')
stock_prices_df = pd.read_csv('./stock_yfinance_data.csv')

# Convert the 'Date' columns to datetime and normalize to just the date
tweets_df['Date'] = pd.to_datetime(tweets_df['Date']).dt.date
stock_prices_df['Date'] = pd.to_datetime(stock_prices_df['Date']).dt.date

# Merge the sentiment data with the stock prices data on the date and stock name
merged_df = pd.merge(stock_prices_df, tweets_df, left_on=['Date', 'Stock Name'], right_on=['Date', 'Stock Name'])

# Group by Date and Stock Name, and then calculate the mean sentiment
average_sentiment_per_day = merged_df.groupby(['Date', 'Stock Name'])[sentiment_type].mean().reset_index()

# Merge the average sentiment back with the stock prices
merged_with_average_sentiment = pd.merge(stock_prices_df, average_sentiment_per_day, on=['Date', 'Stock Name'])

# Filter for a specific stock
start_dates = ['2021-09-30','2022-01-01','2022-06-30','2021-09-30']
end_dates = ['2022-01-01','2022-06-30','2022-09-30','2022-09-30']
for i in merged_with_average_sentiment['Stock Name'].unique():
    for x in range(len(start_dates)):
        specific_stock_df = merged_with_average_sentiment[merged_with_average_sentiment['Stock Name'] == i]

        # Set the date as the index for the specific stock
        specific_stock_df.set_index('Date', inplace=True)

        start_date = start_dates[x]
        end_date = end_dates[x]

        # Convert strings to datetime
        start_date = pd.to_datetime(start_date).date()
        end_date = pd.to_datetime(end_date).date()

        # Filter the DataFrame for the date range
        demo_stock_df = specific_stock_df[(specific_stock_df.index >= start_date) & (specific_stock_df.index <= end_date)]

        # Assuming 'merged_data' is your DataFrame containing the 'Close' and 'positive_sentiment' columns
        # positive_sentiment
        demo_stock_df['positive_sentiment_change'] = demo_stock_df['positive_sentiment'].pct_change()

        
        demo_stock_df['Close_change'] = demo_stock_df['Close'].pct_change()

        
        demo_stock_df = demo_stock_df.dropna()

        change_correlation = demo_stock_df['Close_change'].corr(demo_stock_df['positive_sentiment_change'])

        # positive_sentiment
        demo_stock_df['sentiment_diff'] = demo_stock_df['positive_sentiment'].diff()
        demo_stock_df['close_diff'] = demo_stock_df['Close'].diff()

        
        demo_stock_df['direction_consistent'] = (demo_stock_df['sentiment_diff'] * demo_stock_df['close_diff'] > 0).astype(int)

        
        consistency_percentage = demo_stock_df['direction_consistent'].mean() * 100


        demo_df = tweets_df[(tweets_df['Date'] >= start_date) & (tweets_df['Date'] <= end_date)]
        demo_df = demo_df[demo_df['Stock Name']==i]
        num = demo_df['Tweet'].count()
        name = demo_df['Company Name'].unique()[0]
        print(f'{i} & {name} & from {start_date} to {end_date} & {num} & {consistency_percentage:.2f}\% & {change_correlation:.2f} \\\ ')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


TSLA & Tesla, Inc. & from 2021-09-30 to 2022-01-01 & 11074 & 57.81\% & 0.28 \\ 
TSLA & Tesla, Inc. & from 2022-01-01 to 2022-06-30 & 18851 & 61.79\% & 0.49 \\ 
TSLA & Tesla, Inc. & from 2022-06-30 to 2022-09-30 & 7643 & 50.79\% & 0.26 \\ 
TSLA & Tesla, Inc. & from 2021-09-30 to 2022-09-30 & 37422 & 58.96\% & 0.40 \\ 
MSFT & Microsoft Corporation & from 2021-09-30 to 2022-01-01 & 785 & 53.12\% & 0.15 \\ 
MSFT & Microsoft Corporation & from 2022-01-01 to 2022-06-30 & 2468 & 54.47\% & -0.04 \\ 
MSFT & Microsoft Corporation & from 2022-06-30 to 2022-09-30 & 857 & 53.97\% & 0.33 \\ 
MSFT & Microsoft Corporation & from 2021-09-30 to 2022-09-30 & 4089 & 53.78\% & 0.09 \\ 
PG & Procter & Gamble Company & from 2021-09-30 to 2022-01-01 & 785 & 48.44\% & 0.12 \\ 
PG & Procter & Gamble Company & from 2022-01-01 to 2022-06-30 & 2468 & 47.97\% & 0.04 \\ 
PG & Procter & Gamble Company & from 2022-06-30 to 2022-09-30 & 857 & 47.62\% & -0.06 \\ 
PG & Procter & Gamble Company & from 2021-09-30 to 2022-0

In [1]:
pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Obtaining dependency information for tabulate from https://files.pythonhosted.org/packages/40/44/4a5f08c96eb108af5cb50b41f76142f0afa346dfa99d5296fe7202a11854/tabulate-0.9.0-py3-none-any.whl.metadata
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from tabulate import tabulate

# Initialize a list to hold the rows of the table
table_data = []

for i in merged_with_average_sentiment['Stock Name'].unique():
    for x in range(len(start_dates)):
        specific_stock_df = merged_with_average_sentiment[merged_with_average_sentiment['Stock Name'] == i]

        # Set the date as the index for the specific stock
        specific_stock_df.set_index('Date', inplace=True)

        start_date = start_dates[x]
        end_date = end_dates[x]

        # Convert strings to datetime
        start_date = pd.to_datetime(start_date).date()
        end_date = pd.to_datetime(end_date).date()

        # Filter the DataFrame for the date range
        demo_stock_df = specific_stock_df[(specific_stock_df.index >= start_date) & (specific_stock_df.index <= end_date)]

        # Calculate percentage changes
        demo_stock_df['positive_sentiment_change'] = demo_stock_df['positive_sentiment'].pct_change()
        demo_stock_df['Close_change'] = demo_stock_df['Close'].pct_change()
        demo_stock_df = demo_stock_df.dropna()

        change_correlation = demo_stock_df['Close_change'].corr(demo_stock_df['positive_sentiment_change'])

        # Calculate differences
        demo_stock_df['sentiment_diff'] = demo_stock_df['positive_sentiment'].diff()
        demo_stock_df['close_diff'] = demo_stock_df['Close'].diff()

        # Check direction consistency
        demo_stock_df['direction_consistent'] = (demo_stock_df['sentiment_diff'] * demo_stock_df['close_diff'] > 0).astype(int)
        consistency_percentage = demo_stock_df['direction_consistent'].mean() * 100

        # Count the number of tweets
        demo_df = tweets_df[(tweets_df['Date'] >= start_date) & (tweets_df['Date'] <= end_date)]
        demo_df = demo_df[demo_df['Stock Name'] == i]
        num = demo_df['Tweet'].count()
        name = demo_df['Company Name'].unique()[0]

        # Append the row to the table data
        table_data.append([i, name, f'{start_date} to {end_date}', num, f'{consistency_percentage:.2f}%', f'{change_correlation:.2f}'])

# Define the headers of the table
headers = ['Stock Name', 'Company Name', 'Date Range', 'Number of Tweets', 'Consistency Percentage', 'Change Correlation']

# Print the table with a compact format
print(tabulate(table_data, headers=headers, tablefmt='plain', stralign='left', numalign='right'))


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Stock Name    Company Name                                        Date Range                  Number of Tweets  Consistency Percentage      Change Correlation
TSLA          Tesla, Inc.                                         2021-09-30 to 2022-01-01               11074  57.81%                                    0.28
TSLA          Tesla, Inc.                                         2022-01-01 to 2022-06-30               18851  61.79%                                    0.49
TSLA          Tesla, Inc.                                         2022-06-30 to 2022-09-30                7643  50.79%                                    0.26
TSLA          Tesla, Inc.                                         2021-09-30 to 2022-09-30               37422  58.96%                                     0.4
MSFT          Microsoft Corporation                               2021-09-30 to 2022-01-01                 785  53.12%                                    0.15
MSFT          Microsoft Corporation           